---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [21]:
spam_data.groupby('target').count().loc[1,'text']/len(spam_data)*100

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

In [30]:
token_df = pd.DataFrame(vectorizer.get_feature_names(),columns=['token'])

In [43]:
token_df.loc[(token_df['token'].apply(lambda x:len(x))).idxmax()]['token']

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [236]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

vectorizer = CountVectorizer().fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
NB = MultinomialNB(alpha=0.1).fit(X_train_vectorized,y_train)
roc_auc_score(y_test,NB.predict_proba(X_test_vectorized)[:,1])

0.991545422134696

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer().fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)

In [81]:
tf_idf_series = pd.Series(X_train_vectorized.max(0).toarray()[0], index = vectorizer.get_feature_names())

In [94]:
(tf_idf_series.sort_index(ascending = True).sort_values(ascending=True,kind='mergesort')[:20],
tf_idf_series.sort_index(ascending = False).sort_values(ascending=False,kind='mergesort')[:20])

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [237]:
vectorizer = TfidfVectorizer(min_df=3).fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
NB = MultinomialNB(alpha=0.1).fit(X_train_vectorized,y_train)
roc_auc_score(y_test,NB.predict_proba(X_test_vectorized)[:,1])

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [298]:
import re
length = pd.DataFrame({'length':spam_data.text.apply(lambda x:len(x)),'target':spam_data.target}).groupby('target')['length'].mean().values
(length[0],length[1])

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [123]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [299]:
from sklearn.svm import SVC

vectorizer = TfidfVectorizer(min_df=5).fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

X_train_add_feature = add_feature(X_train_vectorized,pd.DataFrame(X_train.apply(lambda x:len(x))).reset_index(drop=True).T)
X_test_add_feature = add_feature(X_test_vectorized,pd.DataFrame(X_test.apply(lambda x:len(x))).reset_index(drop=True).T)

svc = SVC(C=10000,probability=True)
svc.fit(X_train_add_feature,y_train)
roc_auc_score(y_test,svc.predict_proba(X_test_add_feature)[:,1])

C:\Users\Junbo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9951106055718724

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [300]:
import re
digit = pd.DataFrame({'digit':spam_data.text.apply(lambda x:len(re.findall('\d',x))),'target':spam_data['target']}).groupby('target')['digit'].mean().values
(digit[0],digit[1])

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [301]:
from sklearn.svm import SVC

vectorizer = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

X_train_add_feature = add_feature(X_train_vectorized,
                                  pd.DataFrame({'character_number':X_train.apply(lambda x:len(x)),
                                                'digit_number':X_train.apply(lambda x:len(re.findall('\d',x)))}).reset_index(drop=True).T)
X_test_add_feature = add_feature(X_test_vectorized,
                                  pd.DataFrame({'character_number':X_test.apply(lambda x:len(x)),
                                                'digit_number':X_test.apply(lambda x:len(re.findall('\d',x)))}).reset_index(drop=True).T)

svc = SVC(C=100,probability=True)
svc.fit(X_train_add_feature,y_train)
roc_auc_score(y_test,svc.predict_proba(X_test_add_feature)[:,1])

C:\Users\Junbo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9929248085836034

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [303]:
non_word = pd.DataFrame({'num':spam_data.text.apply(lambda x:len(re.findall('\W',x))),'target':spam_data.target}).reset_index().groupby('target')['num'].mean().values
(non_word[0],non_word[1])

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [304]:
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

X_train_add_feature = add_feature(X_train_vectorized,
                                  pd.DataFrame({'length_of_doc':X_train.apply(lambda x:len(x)),
                                                'digit_count':X_train.apply(lambda x:len(re.findall('\d',x))),
                                                'non_word_char_count':X_train.apply(lambda x:len(re.findall('\W',x)))}).reset_index(drop=True).T)
X_test_add_feature = add_feature(X_test_vectorized,
                                  pd.DataFrame({'length_of_doc':X_test.apply(lambda x:len(x)),
                                                'digit_count':X_test.apply(lambda x:len(re.findall('\d',x))),
                                                'non_word_char_count':X_test.apply(lambda x:len(re.findall('\W',x)))}).reset_index(drop=True).T)

lgr = LogisticRegression(C=100)
lgr.fit(X_train_add_feature,y_train)
coef_series = pd.Series(lgr.coef_[0], index = vectorizer.get_feature_names()+['length_of_doc', 'digit_count', 'non_word_char_count']).sort_values(ascending = True)

C:\Users\Junbo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [309]:
(roc_auc_score(y_test,lgr.predict_proba(X_test_add_feature)[:,1]),
 [coef_series[:10].index],
 [coef_series[:-10:-1].index])

(0.9952506663497614,
 [Index(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], dtype='object')],
 [Index(['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww'], dtype='object')])